# Data Import: City of Chicago Data Portal

## Traffic Data: each record is crash event
   ### https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
## Traffic Data: Each record describes person involved in event
   ### https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d
## Traffic Data: Each Record describs vehicle involved in event
   ### https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3

### Import Libraries
   #### Socrata is libary needed to access Chicago API

In [2]:
import pandas as pd
import requests
import json
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from sodapy import Socrata
from cdp_secrets import token
print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:41:36.714297


## Chicago Data Portal Connection API Build

In [3]:
#Constants
chicago_url="data.cityofchicago.org"
vehicle_api_root="68nd-jvt3"
people_api_root="u6pd-qa9d"
crash_api_root="85ca-t3if"
district_root="z8bn-74gv"
beat_root="n9it-hstw"
crime_root="ijzp-q8t2"
ca_root="igwz-8jzy"

#Chicago Data Portal Connection
cdp = Socrata(chicago_url,
                 token
             )
print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:41:36.739385


In [4]:
def callAPI(root: str, filter: str) -> pd.DataFrame:
    results=cdp.get(root, query=filter)
    df=pd.DataFrame.from_records(results)
    return df
print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:41:36.763176


In [5]:
# Get dates for the last 60 months
today = datetime.now().strftime("%Y-%m-%d")
eighteen_months = (datetime.now() - relativedelta(months=60)).strftime("%Y-%m-%d")

print(today)
print(eighteen_months)
print('completed at ' + str(datetime.now()))

2024-02-14
2019-02-14
completed at 2024-02-14 20:41:36.777389


In [6]:
#limit required for SoQL query, as we are filtering for one year limit is set high to capture all records
limit='1000000000'

#Filters use SQL structure, some where clauses use a different format. Refer to API Documentation for correct syntax
vehicle_filter= """Select crash_record_id,crash_date,make,model,vehicle_type 
                    where vehicle_type is not null and crash_date between '%s' and '%s'
                    limit %s"""%(eighteen_months,today,limit) 

people_filter= """Select person_id,crash_record_id,crash_date,person_type,age,sex,
                injury_classification,pedpedal_action,pedpedal_visibility,pedpedal_location
                Where CRASH_DATE between '%s' and '%s' and
                (person_type='PEDESTRIAN' or person_type='BICYCLE') limit %s"""% (eighteen_months,today,limit)

crash_filter="""Select *
                where crash_date between '%s' and '%s' limit %s"""% (eighteen_months,today,limit)

district_filter="""Select distinct district,district_name limit 1000"""

beat_filter="""Select beat_num,district limit 300"""

ca_filter="""Select distinct area_numbe, community"""

crime_filter="""Select distinct beat, ward, community_area, district limit %s"""% (limit)

print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:41:36.811527


In [7]:
#People Table
people_df=callAPI(people_api_root,people_filter)

print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:41:38.626372


In [9]:
#Vehicle Table
vehicle_df=callAPI(vehicle_api_root,vehicle_filter)

print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:42:17.709168


In [10]:
#Crash Table
crash_df=callAPI(crash_api_root,crash_filter)

print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:44:11.570121


In [11]:
ca_df=callAPI(ca_root,ca_filter)

In [12]:
#crime_table
crime_df=callAPI(crime_root,crime_filter)

print('completed at ' + str(datetime.now()))

completed at 2024-02-14 20:44:30.766286


In [13]:
#Beat To District Key Table
#To be Replaced with neighborhood table

#Police Beat
beat_df=callAPI(beat_root,beat_filter)

#Remove leading 0 from district field
beat_df['district']=pd.to_numeric(beat_df['district'])
beat_df['district']=beat_df['district'].apply(lambda x: str(x))

#Remove leading 0 from beat_num field
beat_df['beat_num']=pd.to_numeric(beat_df['beat_num'])
beat_df['beat_num']=beat_df['beat_num'].apply(lambda x: str(x))

#District table
district_df=callAPI(district_root,district_filter)

#join together to create matching table
district_key=pd.merge(beat_df,district_df,on='district',how='left')

print(district_key)
print('completed at ' + str(datetime.now()))

    beat_num district   district_name
0       1713       17     Albany Park
1       3100       31             NaN
2       1651       16  Jefferson Park
3       1914       19       Town Hall
4       1915       19       Town Hall
..       ...      ...             ...
272      314        3  Grand Crossing
273      825        8    Chicago Lawn
274      313        3  Grand Crossing
275      823        8    Chicago Lawn
276      312        3  Grand Crossing

[277 rows x 3 columns]
completed at 2024-02-14 20:44:33.117736


In [14]:
df = pd.merge(crime_df, district_key, left_on='beat', right_on='beat_num', how='left')
df= pd.merge(df,ca_df, left_on='community_area',right_on='area_numbe', how='left')
ca_beat_key=df
ca_beat_key

,beat,ward,community_area,district_x,beat_num,district_y,district_name,area_numbe,community
0,0431,10,51,004,NaN,NaN,NaN,51,SOUTH DEERING
1,1631,36,17,031,1631,16,Jefferson Park,17,DUNNING
2,1611,39,10,016,1611,16,Jefferson Park,10,NORWOOD PARK
3,0312,NaN,NaN,003,NaN,NaN,NaN,NaN,NaN
4,0532,10,54,005,NaN,NaN,NaN,54,RIVERDALE
...,...,...,...,...,...,...,...,...,...
2411,2132,4,39,002,NaN,NaN,NaN,39,KENWOOD
2412,1414,35,NaN,014,1414,14,Shakespeare,NaN,NaN
2413,0332,NaN,NaN,003,NaN,NaN,NaN,NaN,NaN
2414,1412,NaN,NaN,014,1412,14,Shakespeare,NaN,NaN


In [15]:

#Join People and Vehicle data sets on crash_record_id
people_vehicle=pd.merge(people_df,vehicle_df[['crash_record_id','make','model','vehicle_type']], on='crash_record_id', how='left')
#print(people_vehicle)

#Join prior dataset with crash information
crash_pv=pd.merge(people_vehicle,crash_df, on='crash_record_id', how='left')
#print(crash_pv)

#Join prior dataset with district ID and Name
#merge_df=pd.merge(crash_pv,district_key, on='beat_num', how='left')
merge_df = pd.merge(crash_pv, district_key, left_on='beat_of_occurrence', right_on='beat_num', how='left')
#Add Neighborhood Name
merger_df1=pd.merge(merge_df,ca_beat_key, left_on='beat_of_occurrence',right_on='beat',how='left')
#Declare final data model
model=merger_df1


#print(model)
print("Model Field List \n")
n=1
for fields in model.columns:
    print(str(n)+'.'+fields)
    n+=1

print('completed at ' + str(datetime.now()))

Model Field List 

1.person_id
2.crash_record_id
3.crash_date_x
4.person_type
5.age
6.sex
7.injury_classification
8.pedpedal_action
9.pedpedal_visibility
10.pedpedal_location
11.make
12.model
13.vehicle_type
14.crash_date_y
15.posted_speed_limit
16.traffic_control_device
17.device_condition
18.weather_condition
19.lighting_condition
20.first_crash_type
21.trafficway_type
22.alignment
23.roadway_surface_cond
24.road_defect
25.report_type
26.crash_type
27.damage
28.date_police_notified
29.prim_contributory_cause
30.sec_contributory_cause
31.street_no
32.street_direction
33.street_name
34.beat_of_occurrence
35.num_units
36.most_severe_injury
37.injuries_total
38.injuries_fatal
39.injuries_incapacitating
40.injuries_non_incapacitating
41.injuries_reported_not_evident
42.injuries_no_indication
43.injuries_unknown
44.crash_hour
45.crash_day_of_week
46.crash_month
47.latitude
48.longitude
49.location
50.intersection_related_i
51.hit_and_run_i
52.photos_taken_i
53.statements_taken_i
54.crash_d

In [16]:
model

,person_id,crash_record_id,crash_date_x,person_type,age,sex,injury_classification,pedpedal_action,pedpedal_visibility,pedpedal_location,...,district_name_x,beat,ward,community_area,district_x,beat_num_y,district_y,district_name_y,area_numbe,community
0,O1758841,361aa34b5e35faf3f36c2dc2d1190d0744af512bc9f2d2...,2024-02-13T18:03:00.000,PEDESTRIAN,43,F,NONINCAPACITATING INJURY,UNKNOWN/NA,NO CONTRASTING CLOTHING,IN ROADWAY,...,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O1758846,8a8191de35790dd48e4bec4d229d35c520202ab8de8e07...,2024-02-13T17:00:00.000,PEDESTRIAN,37,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),CONTRASTING CLOTHING,IN ROADWAY,...,Lincoln,2013,NaN,NaN,020,2013,20,Lincoln,NaN,NaN
2,O1758846,8a8191de35790dd48e4bec4d229d35c520202ab8de8e07...,2024-02-13T17:00:00.000,PEDESTRIAN,37,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),CONTRASTING CLOTHING,IN ROADWAY,...,Lincoln,2013,48,77,020,2013,20,Lincoln,77,EDGEWATER
3,O1758846,8a8191de35790dd48e4bec4d229d35c520202ab8de8e07...,2024-02-13T17:00:00.000,PEDESTRIAN,37,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),CONTRASTING CLOTHING,IN ROADWAY,...,Lincoln,2013,48,77,025,2013,20,Lincoln,77,EDGEWATER
4,O1758846,8a8191de35790dd48e4bec4d229d35c520202ab8de8e07...,2024-02-13T17:00:00.000,PEDESTRIAN,37,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),CONTRASTING CLOTHING,IN ROADWAY,...,Lincoln,2013,48,77,008,2013,20,Lincoln,77,EDGEWATER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132349,O584028,b78d2743ebbd8f8eeb68238abfba43894f30919729511b...,2019-02-14T01:05:00.000,PEDESTRIAN,NaN,M,INCAPACITATING INJURY,NOT AT INTERSECTION,NO CONTRASTING CLOTHING,IN ROADWAY,...,Harrison,1113,28,26,011,1113,11,Harrison,26,WEST GARFIELD PARK
132350,O584028,b78d2743ebbd8f8eeb68238abfba43894f30919729511b...,2019-02-14T01:05:00.000,PEDESTRIAN,NaN,M,INCAPACITATING INJURY,NOT AT INTERSECTION,NO CONTRASTING CLOTHING,IN ROADWAY,...,Harrison,1113,28,25,015,1113,11,Harrison,25,AUSTIN
132351,O584028,b78d2743ebbd8f8eeb68238abfba43894f30919729511b...,2019-02-14T01:05:00.000,PEDESTRIAN,NaN,M,INCAPACITATING INJURY,NOT AT INTERSECTION,NO CONTRASTING CLOTHING,IN ROADWAY,...,Harrison,1113,28,25,011,1113,11,Harrison,25,AUSTIN
132352,O584028,b78d2743ebbd8f8eeb68238abfba43894f30919729511b...,2019-02-14T01:05:00.000,PEDESTRIAN,NaN,M,INCAPACITATING INJURY,NOT AT INTERSECTION,NO CONTRASTING CLOTHING,IN ROADWAY,...,Harrison,1113,28,25,NaN,1113,11,Harrison,25,AUSTIN


In [17]:
#remove duplicate records where person_id is the same
model=model.drop_duplicates(subset='person_id',keep='first')

In [18]:
model

,person_id,crash_record_id,crash_date_x,person_type,age,sex,injury_classification,pedpedal_action,pedpedal_visibility,pedpedal_location,...,district_name_x,beat,ward,community_area,district_x,beat_num_y,district_y,district_name_y,area_numbe,community
0,O1758841,361aa34b5e35faf3f36c2dc2d1190d0744af512bc9f2d2...,2024-02-13T18:03:00.000,PEDESTRIAN,43,F,NONINCAPACITATING INJURY,UNKNOWN/NA,NO CONTRASTING CLOTHING,IN ROADWAY,...,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O1758846,8a8191de35790dd48e4bec4d229d35c520202ab8de8e07...,2024-02-13T17:00:00.000,PEDESTRIAN,37,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),CONTRASTING CLOTHING,IN ROADWAY,...,Lincoln,2013,NaN,NaN,020,2013,20,Lincoln,NaN,NaN
5,O1758722,8b92e4deb0ded15a89246d7bbafcf26d21f533e08f2462...,2024-02-13T15:00:00.000,PEDESTRIAN,14,F,NONINCAPACITATING INJURY,CROSSING - NO CONTROLS (NOT AT INTERSECTION),NO CONTRASTING CLOTHING,IN CROSSWALK,...,Ogden,1024,12,30,010,1024,10,Ogden,30,SOUTH LAWNDALE
11,O1758723,8b92e4deb0ded15a89246d7bbafcf26d21f533e08f2462...,2024-02-13T15:00:00.000,PEDESTRIAN,5,F,"REPORTED, NOT EVIDENT",CROSSING - NO CONTROLS (NOT AT INTERSECTION),NO CONTRASTING CLOTHING,IN CROSSWALK,...,Ogden,1024,12,30,010,1024,10,Ogden,30,SOUTH LAWNDALE
17,O1758649,854c554ab119d07586872564bd5d2b5c2d2137f120157f...,2024-02-13T12:55:00.000,BICYCLE,NaN,M,NO INDICATION OF INJURY,CROSSING - CONTROLS PRESENT (NOT AT INTERSECTION),NO CONTRASTING CLOTHING,IN ROADWAY,...,Chicago Lawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132334,O584397,fd3b400aeb7b889a99ac1151cdc8285ac2406ca2fe3d2e...,2019-02-14T13:15:00.000,PEDESTRIAN,18,F,NONINCAPACITATING INJURY,PARKED VEHICLE,NO CONTRASTING CLOTHING,IN ROADWAY,...,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132335,O584293,fe7d3a8955057cdf8a2ef30a22d9127b73beabbd49b7cb...,2019-02-14T08:39:00.000,BICYCLE,38,M,NONINCAPACITATING INJURY,NO ACTION,CONTRASTING CLOTHING,BIKEWAY,...,Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132336,O584196,36128d6af5bb6f1db84495d8168cd6b78a087139bf9335...,2019-02-14T08:35:00.000,PEDESTRIAN,27,F,NONINCAPACITATING INJURY,CROSSING - WITH SIGNAL,NO CONTRASTING CLOTHING,IN CROSSWALK,...,Near North,1834,42,8,001,1834,18,Near North,8,NEAR NORTH SIDE
132345,O584459,b4e6026f6b30931534ded4a656e6d061b7f9ff0bd319a7...,2019-02-14T06:35:00.000,PEDESTRIAN,26,M,NONINCAPACITATING INJURY,NO ACTION,NO CONTRASTING CLOTHING,IN CROSSWALK,...,Near North,1833,2,8,018,1833,18,Near North,8,NEAR NORTH SIDE


In [19]:
model
model.to_excel('pedestrian_incidents.xlsx',index=True)
print('completed at '+str(datetime.now()))

completed at 2024-02-14 20:45:46.853681
